In [11]:
import os 
import re
import matplotlib.pyplot as plt
import rmgpy.chemkin
import numpy as np
import cantera as ct
import random
import pandas as pd
%matplotlib inline

Let's get mapping of between RMG models and NIST

In [12]:
###### load chemkin files

#RMG 204 (smaller 182 species should also have the same species and reaction numbers)
full_path_RMG = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_with_BROH'
chemkin_path_RMG = os.path.join(full_path_RMG,'chemkin/copies/copy_chem_annotated_204species.inp')
transport_path_RMG = os.path.join(full_path_RMG, 'chemkin/copies/tran.dat')
dictionary_path_RMG = os.path.join(full_path_RMG,'chemkin/species_dictionary.txt')

RMG_species, RMG_reactions = rmgpy.chemkin.load_chemkin_file(chemkin_path_RMG, dictionary_path=dictionary_path_RMG, transport_path=transport_path_RMG)

#NIST model 
full_path_NIST = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/NIST'
chemkin_path_NIST = os.path.join(full_path_NIST,'2-BTP_kinetics_with_M.inp')
dictionary_path_NIST = os.path.join(full_path_NIST,'species_dictionary-2_BTP.txt')

NIST_species, NIST_reactions = rmgpy.chemkin.load_chemkin_file(chemkin_path_NIST, dictionary_path=dictionary_path_NIST)


In [3]:
#model 1 = RMG, model 2 = NIST


In [14]:
def compare_rxns(model_1_reactions, model_2_reactions): 
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (model_1_reactions, model_2_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    if len(to_remove)!=0:
        for reactionList, reaction in to_remove:
            reactionList.remove(reaction)
        print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = {}
    common_reactions_equations = {}
    unique_reactions_1 = []
    unique_reactions_2 = []
    copy_of_model_1_reactions = model_1_reactions.copy() #list 1
    copy_of_model_2_reactions = model_2_reactions.copy() #list 2

    
    for model_2_rxn in copy_of_model_2_reactions:
        for model_1_rxn in copy_of_model_1_reactions: # make a copy so you don't remove from the list you are iterating over
            counter = 0 #use this to see how many times this loop below was executed
            if model_2_rxn.is_isomorphic(model_1_rxn):
                common_reactions[model_1_rxn]= model_2_rxn
                common_reactions_equations[str(model_1_rxn)]= str(model_2_rxn)
                copy_of_model_1_reactions.remove(model_1_rxn)
                counter += 1
            if counter>1:
                print([str(model_1_rxn),str(model_2_rxn)]) #let's see if a rxn has two matches
                
 
    #find the unique reactions of each model
    for model_2_rxn in copy_of_model_2_reactions:
        for r1, r2 in common_reactions.items():
            if model_2_rxn is r2:
            #if model_2_rxn.is_isomorphic(r2):
                break
        else:
            unique_reactions_2.append(model_2_rxn)
    for model_1_rxn in copy_of_model_1_reactions:
        for r1, r2 in common_reactions.items():
            if model_1_rxn is r1:
#             if model_1_rxn.is_isomorphic(r1):
                break
        else:
            unique_reactions_1.append(model_1_rxn)
            
    
    #find the reactions that have different kinetics (aren't identical and aren't similar)
    
    different_kinetics_reactions = []
    
    for r1, r2 in common_reactions.items(): 
        if (r2.kinetics.is_identical_to(r1.kinetics)==False) and (r2.kinetics.is_similar_to(r1.kinetics)==False):
            different_kinetics_reactions.append((r1, r2))
            
            
    #compare species
    common_species = {} #RMG = key, NIST = value
    unique_species_RMG = []
    unique_species_NIST = []

    #find common species
    for r in RMG_species: 
        for n in NIST_species:
            if r.is_identical(n,strict=False) or r.is_isomorphic(n, strict=False):
                common_species[r.to_chemkin()] = n.label

    #put unique species into their individual lists
    for r in RMG_species:
        if r not in common_species.keys():
            unique_species_RMG.append(r.label)  
    for n in NIST_species:
        if n not in common_species.values():
            unique_species_NIST.append(n.label)


            
            
    return different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactions_1, unique_reactions_2, common_species, unique_species_RMG, unique_species_NIST
    
    

In [15]:
different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactions_1, unique_reactions_2, common_species, unique_species_RMG, unique_species_NIST = compare_rxns(RMG_reactions, NIST_reactions)

Load the sensitivites

In [16]:
#let's only look at the top 25 reactions 
n = 25 

btps_moles = list(np.linspace(0.0, 0.16268, 10))

#iterate over all the BTP mole fractions:
for value in btps_moles: 
    
    if value == 0.0:
        continue
    
    #volume fraction of BTP = x/(x+1+3.76+0.5). when BTP vol frac is 3%, moles of BTP (normalized to O2) is 0.16268
    vf = value/(value+1+3.76+0.5)*100
    

    #RMG 182 species 
    df_182 = pd.read_csv(f'/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/RMG_with_BROH/182_species/copy_chem0182.cti_{value}_sensitivities.csv')

    #RMG 204 species
    df_204 = pd.read_csv(f'/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/RMG_with_BROH/204_species/copy_chem0204.cti_{value}_sensitivities.csv')

    #NIST model 
    df_NIST = pd.read_csv(f'/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/NIST/2-BTP_kinetics_with_M.cti_{value}_sensitivities.csv')



    ############# iterate over rows of csv file. 
    
    #dictionary to store info about reaction from CSV file 
#     _182_dict = {}
#     _204_dict = {}
#     _NIST_dict = {}
    
    
    #list to store sets containing the reactants and products of each reaction.  
    _182 = []
    _204 = []
    _NIST = []
    
    
    for index,row in df_182.head(n).iterrows(): #Only iterate over the first n rows
        sensitivity, ct_equation, cantera_products, cantera_reactants = row['sensitivity'], row['cantera equation'], row['cantera products'], row['cantera reactants']
        
        #combine the products and reactants into one dictionary
        species_involved = {**eval(cantera_products), **eval(cantera_reactants)}
        
        #turn this into a set that we will compare to NIST reactions
        spcs_involved = set([spcs for spcs in species_involved.keys()])
        
        
        
#         _182_dict[index] = [sensitivity, ct_equation, cantera_products, cantera_reactants]
        _182.append(spcs_involved)
        
    for index,row in df_204.head(n).iterrows(): #Only iterate over the first n rows
        
        sensitivity, ct_equation, cantera_products, cantera_reactants = row['sensitivity'], row['cantera equation'], row['cantera products'], row['cantera reactants']
        #combine the products and reactants into one dictionary
        species_involved = {**eval(cantera_products), **eval(cantera_reactants)}
        
        #turn this into a set that we will compare to NIST reactions
        spcs_involved = set([spcs for spcs in species_involved.keys()])
        
        
        
#         _204_dict[index] = [sensitivity, ct_equation, cantera_products, cantera_reactants]
        _204.append(spcs_involved)
        
    for index,row in df_NIST.head(n).iterrows(): #Only iterate over the first n rows
        sensitivity, ct_equation, cantera_products, cantera_reactants = row['sensitivity'], row['cantera equation'], row['cantera products'], row['cantera reactants']
        #combine the products and reactants into one dictionary
        species_involved = {**eval(cantera_products), **eval(cantera_reactants)}
        
        #turn this into a set that we will compare to NIST reactions
        spcs_involved = set([spcs for spcs in species_involved.keys()])
        
        
        
#         _NIST_dict[index] = [sensitivity, ct_equation, cantera_products, cantera_reactants]
        _NIST.append(spcs_involved)
        
        
        
    print('************************************************************************')
    print(vf)
    #let's start with 204 and compare to 184 species
    
    #make a set of sets
    _182_ = set(frozenset(i) for i in _182)
    _204_ = set(frozenset(i) for i in _204)

    #Of the n top most sensitive reactions in the two models, 
    #this is a set of the reactions that were most sensitive in one model but not the other for each volume fraction. 
    unique_rxns =  _182_.symmetric_difference(_204_) 
    
    if unique_rxns != set():
        #just to print them without set(...)
        unfrozen = [list(i) for i in unique_rxns]
        for i in unfrozen:
            print(i)
            
            
    #as we can see, there wasn't too many reactions that changed in sensitivity across the 182 v. 204 RMG model.  

************************************************************************
0.3424648882968287
************************************************************************
0.6825921381900819
['H(9)', 'HBR(93)', 'OH(2)', 'BROH(8)']
['OH(2)', 'CH3(20)', 'CH3OH(27)']
************************************************************************
1.020405603073009
************************************************************************
1.3559288129038278
************************************************************************
1.6891849796691407
************************************************************************
2.020197002736977
************************************************************************
2.3489874741020595
************************************************************************
2.675578683525816
************************************************************************
2.9999926235735836
['HBR(93)', 'HO2(14)', 'BR(91)', 'O2(4)']
['H(9)', 'OH(2)', 'H2(11)', 'O(10)']


In [30]:
#now let's look at the difference between NIST and RMG (204 species)

#dictionary of sensitive unique RMG reactions for each volume fraction of 2_BTP
vf_RMG_sensitivities = dict()


#top _n_ sensitive reactions
n = 25 

btps_moles = list(np.linspace(0.0, 0.16268, 10))

#iterate over all the BTP mole fractions:
for value in btps_moles: 
    
    if value == 0.0:
        continue
    
    #volume fraction of BTP = x/(x+1+3.76+0.5). when BTP vol frac is 3%, moles of BTP (normalized to O2) is 0.16268
    vf = value/(value+1+3.76+0.5)*100
    

    #RMG 204 species
    df_204 = pd.read_csv(f'/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/RMG_with_BROH/204_species/copy_chem0204.cti_{value}_sensitivities.csv')

    #NIST model 
    df_NIST = pd.read_csv(f'/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/NIST/2-BTP_kinetics_with_M.cti_{value}_sensitivities.csv')



    ############# iterate over rows of csv file. 
    
    #dictionary to store all of the reaction data from csv file
    _204_dict = {}
    _NIST_dict = {}
    
    
    #list to store sets that contain the reactions and products of each reaction that is sensitive. Can't put this in dictionary because can't use sets as keys
    _204 = []
    _NIST = []
    
    
        
    for index,row in df_204.head(n).iterrows(): #Only iterate over the first n rows
        
        sensitivity, ct_equation, cantera_products, cantera_reactants, cantera_index = row['sensitivity'], row['cantera equation'], row['cantera products'], row['cantera reactants'], row['cantera_index']
        
        #combine the products and reactants into one dictionary
        species_involved = {**eval(cantera_products), **eval(cantera_reactants)}

        #since we are comparing RMG species sets to NIST species sets for each reaction, we need to try and convert them to one model's naming scheme.
        species_involved_with_NIST_names = []
        
        for spcs in species_involved.keys():
            try: 
                #if the species involved have NIST equivalents
                species_involved_with_NIST_names.append(common_species[spcs])
                
            except:
                #if they don't, leave them as they were 
                species_involved_with_NIST_names.append(spcs)
        
        
        
        #turn the dictionary into a set that we will compare to NIST reactions
        spcs_involved = set([spcs for spcs in species_involved_with_NIST_names])
      
        
        
        _204_dict[str(spcs_involved)] = [sensitivity, ct_equation, cantera_products, cantera_reactants, cantera_index]
        _204.append(spcs_involved)
        
    for index,row in df_NIST.head(n).iterrows(): #Only iterate over the first n rows
        sensitivity, ct_equation, cantera_products, cantera_reactants, cantera_index =  row['sensitivity'], row['cantera equation'], row['cantera products'], row['cantera reactants'], row['cantera_index']
        #combine the products and reactants into one dictionary
        species_involved = {**eval(cantera_products), **eval(cantera_reactants)}
        
        #turn this into a set that we will compare to NIST reactions
        spcs_involved = set([spcs for spcs in species_involved.keys()])
        
        
        
        _NIST_dict[str(spcs_involved)] = [sensitivity, ct_equation, cantera_products, cantera_reactants, cantera_index]
        _NIST.append(spcs_involved)
    
    ########## comparison ######################
    
    print('************************************************************************')
    print(f'Volume fraction of 2_BTP: {vf}')
    print('\n')
    
    
    #make a set of sets for each model's most sensitive reactions reported in the csv files
    _NIST_ = set([frozenset(i) for i in _NIST])
    _204_ = set([frozenset(i) for i in _204])

    
    #find the differences among the most sensitive reactions between the NIST and RMG model 
    unique_rxns =  _204_.symmetric_difference(_NIST_) 

    
    #will be added to vf_RMG_sensitivities as a value for this volume fraction
    RMG_uniq_sens = []
    
    #if there is a difference, show me it
    if unique_rxns != set():
            
                
        for i in unique_rxns:
            if i in _204_:
                
                #might have to shuffle the set to match it to the key in the dictionary. Should have probably just used a frozen set as a key for dictionary?  
                count = 0
                while count ==0:
                    try: 
                        y = set(i)
                        print( "\x1b[34mRMG\x1b[0m", '\t', f'\x1b[34m{i}\x1b[0m', '\t\t\t\t', abs(_204_dict[str(y)][0]) )  #ANSI Color Escape codes
                        RMG_uniq_sens.append((_204_dict[str(y)][-1],_204_dict[str(y)][1],_204_dict[str(y)][0]))
                        count = 1
                    except KeyError:
                        list_of_set = [x for x in y]
                        random.shuffle(list_of_set)
                        y = set(list_of_set)
                        try:
                            print( "\x1b[34mRMG\x1b[0m", '\t', f'\x1b[34m{i}\x1b[0m', '\t\t\t\t', abs(_204_dict[str(y)][0]) )  #ANSI Color Escape codes
                            RMG_uniq_sens.append((_204_dict[str(y)][-1],_204_dict[str(y)][1],_204_dict[str(y)][0]))
                            count = 1
                        except:
                            break   
                        
            if i in _NIST_:
                count = 0
                while count ==0:
                    try: 
                        y = set(i)
                        print( "NIST", '\t', i, '\t\t\t\t', abs(_NIST_dict[str(y)][0]) )  #ANSI Color Escape codes
                        count = 1
                    except KeyError:
                        list_of_set = [x for x in y]
                        random.shuffle(list_of_set)
                        y = set(list_of_set)
                        try:
                            print( "NIST", '\t', i, '\t\t\t\t', abs(_NIST_dict[str(y)][0]) )  #ANSI Color Escape codes
                            count = 1
                        except:
                            break

                
                
    vf_RMG_sensitivities[vf] = RMG_uniq_sens         
    print(len(unique_rxns))
            

************************************************************************
Volume fraction of 2_BTP: 0.3424648882968287


NIST 	 frozenset({'CH', 'H2', 'H', 'CH2'}) 				 0.06867194478330937
RMG 	 frozenset({'O2', 'CH2', 'CHO2(230)', 'H'}) 				 0.013528403550981777
NIST 	 frozenset({'CH3', 'CH4', 'H'}) 				 0.09684759665072466
RMG 	 frozenset({'OH', 'BROH', 'BR'}) 				 0.02386866725726913
NIST 	 frozenset({'H2O', 'CO', 'HCO', 'H'}) 				 0.14721916374818228
RMG 	 frozenset({'CH3', 'OH', 'CH4', 'O'}) 				 0.0061382116383421155
NIST 	 frozenset({'CH2*', 'CO', 'OH', 'H', 'O2'}) 				 0.13102653310061502
NIST 	 frozenset({'H2', 'OH', 'O', 'H'}) 				 0.10669569814686417
NIST 	 frozenset({'OH', 'O2', 'HCO', 'CH2'}) 				 0.11055990973894016
NIST 	 frozenset({'H2O', 'OH', 'H'}) 				 0.0901713855237037
RMG 	 frozenset({'OH', 'O', 'BR', 'HBR'}) 				 0.010348150163783171
RMG 	 frozenset({'C2H2', 'O', 'HCCO', 'H'}) 				 0.0063595441322857145
RMG 	 frozenset({'H2O', 'CO', 'OH', 'HCO'}) 				 0.0142219478

These are the most sensitive reactions in the NIST or RMG model. 
If it is in blue, that means it was among the most sensitive reactions for RMG but not for NIST. 
If it is in black, it was among the most sensitive reactions for NIST but not for RMG.

Plan: Compile the RMG reactions in blue among all the volume fractions, then either switch their kinetics to NIST kinetics or delete the reaction if not in NIST. Do this with different "tolerances" of sensitivities. 

In [31]:
#Below are the compiled RMG reactions in blue, at each volume fraction: 

vf_RMG_sensitivities

{0.3424648882968287: [(996,
   'CH2(T)(19) + O2(4) <=> CHO2(230) + H(9)',
   -0.013528403550981777),
  (411, 'BROH(8) (+M) <=> BR(91) + OH(2) (+M)', 0.02386866725726913),
  (123, 'CH4(3) + O(10) <=> CH3(20) + OH(2)', 0.0061382116383421155),
  (776, 'HBR(93) + O(10) <=> BR(91) + OH(2)', -0.010348150163783171),
  (162, 'C2H2(24) + O(10) <=> H(9) + HCCO(22)', -0.0063595441322857145),
  (36, 'HCO(18) + OH(2) <=> CO(16) + H2O(5)', -0.014221947886390897),
  (412, 'BR(91) + HO2(14) <=> HBR(93) + O2(4)', 0.015152354764004504),
  (3, 'H2(11) + OH(2) <=> H(9) + H2O(5)', 0.005556895681581534),
  (103, '2 CH3(20) <=> C2H5(33) + H(9)', -0.0042928084743505965),
  (1160, 'CHF2(83) + O2(4) <=> CF2O(50) + OH(2)', -0.012281918829513816),
  (1210, 'CHF2(83) + HCO(18) <=> CH2F2(42) + CO(16)', 0.005811494361028936),
  (717, 'H(9) + HBR(93) <=> BR(91) + H2(11)', 0.017840354082740966),
  (381, 'CH2O(21) + H(9) <=> CH2OH(34)', -0.005294849055472423)],
 0.6825921381900819: [(996,
   'CH2(T)(19) + O2(4) <=> CHO

In [32]:
#let's get rid of the duplicates and have one list across as volume fractions 
RMG_indices_no_duplicates = []
indices = []

for value in vf_RMG_sensitivities.values(): 
    print(value)
    for index, eq, sens in value: 
        if index not in indices:
            indices.append(index)
            RMG_indices_no_duplicates.append((index, eq, abs(sens)))

[(996, 'CH2(T)(19) + O2(4) <=> CHO2(230) + H(9)', -0.013528403550981777), (411, 'BROH(8) (+M) <=> BR(91) + OH(2) (+M)', 0.02386866725726913), (123, 'CH4(3) + O(10) <=> CH3(20) + OH(2)', 0.0061382116383421155), (776, 'HBR(93) + O(10) <=> BR(91) + OH(2)', -0.010348150163783171), (162, 'C2H2(24) + O(10) <=> H(9) + HCCO(22)', -0.0063595441322857145), (36, 'HCO(18) + OH(2) <=> CO(16) + H2O(5)', -0.014221947886390897), (412, 'BR(91) + HO2(14) <=> HBR(93) + O2(4)', 0.015152354764004504), (3, 'H2(11) + OH(2) <=> H(9) + H2O(5)', 0.005556895681581534), (103, '2 CH3(20) <=> C2H5(33) + H(9)', -0.0042928084743505965), (1160, 'CHF2(83) + O2(4) <=> CF2O(50) + OH(2)', -0.012281918829513816), (1210, 'CHF2(83) + HCO(18) <=> CH2F2(42) + CO(16)', 0.005811494361028936), (717, 'H(9) + HBR(93) <=> BR(91) + H2(11)', 0.017840354082740966), (381, 'CH2O(21) + H(9) <=> CH2OH(34)', -0.005294849055472423)]
[(996, 'CH2(T)(19) + O2(4) <=> CHO2(230) + H(9)', -0.030410634213313286), (116, 'CH2OH(34) + H(9) <=> CH3(20) 

In [34]:
RMG_indices_no_duplicates

#This is a list of tuples. First value in tuple is the cantera_index of the unique RMG reaction, second is the RMG equation, third is the sensitivity.
#can use this sensitivity to create a "tolerance"

[(996, 'CH2(T)(19) + O2(4) <=> CHO2(230) + H(9)', 0.013528403550981777),
 (411, 'BROH(8) (+M) <=> BR(91) + OH(2) (+M)', 0.02386866725726913),
 (123, 'CH4(3) + O(10) <=> CH3(20) + OH(2)', 0.0061382116383421155),
 (776, 'HBR(93) + O(10) <=> BR(91) + OH(2)', 0.010348150163783171),
 (162, 'C2H2(24) + O(10) <=> H(9) + HCCO(22)', 0.0063595441322857145),
 (36, 'HCO(18) + OH(2) <=> CO(16) + H2O(5)', 0.014221947886390897),
 (412, 'BR(91) + HO2(14) <=> HBR(93) + O2(4)', 0.015152354764004504),
 (3, 'H2(11) + OH(2) <=> H(9) + H2O(5)', 0.005556895681581534),
 (103, '2 CH3(20) <=> C2H5(33) + H(9)', 0.0042928084743505965),
 (1160, 'CHF2(83) + O2(4) <=> CF2O(50) + OH(2)', 0.012281918829513816),
 (1210, 'CHF2(83) + HCO(18) <=> CH2F2(42) + CO(16)', 0.005811494361028936),
 (717, 'H(9) + HBR(93) <=> BR(91) + H2(11)', 0.017840354082740966),
 (381, 'CH2O(21) + H(9) <=> CH2OH(34)', 0.005294849055472423),
 (116, 'CH2OH(34) + H(9) <=> CH3(20) + OH(2)', 0.011548326359040208),
 (758, 'BROH(8) + H(9) <=> HBR(93) 

In [41]:
#let's make a list of sensitivity tolerances, and then change them manually bc im too lazy to make a script to do it
tolerances=[0.02, 0.015, 0.01, 0]

for tol in tolerances:
    print("*********************************************************************")
    print(tol)
    for index, eq, sens in RMG_indices_no_duplicates: 
        if sens > tol:    
            print(index, eq, sens)
            


*********************************************************************
0.02
411 BROH(8) (+M) <=> BR(91) + OH(2) (+M) 0.02386866725726913
90 CH3(20) + O(10) <=> CH2O(21) + H(9) 0.03338504523418321
32 HCO(18) + M <=> CO(16) + H(9) + M 0.05294739820867874
1284 C3H3(6704) + HBR(93) <=> BR(91) + C#CC(8822) 0.023219161173965684
1286 C3H3(6704) + H(9) <=> C#CC(8822) 0.026317137007261075
1322 C2H2(24) + CH3(20) <=> C#CC(8822) + H(9) 0.02235247311521645
657 BR(91) + CH3(20) <=> CH2(T)(19) + HBR(93) 0.026156550697682374
560 BR(91) + CH3(20) <=> CBr(425) 0.04317511165651759
512 O2(4) + S(127) <=> O(10) + S(1329) 0.02345517107391097
164 C2H2(24) + OH(2) <=> CH2CO(29) + H(9) 0.024548270552658968
*********************************************************************
0.015
411 BROH(8) (+M) <=> BR(91) + OH(2) (+M) 0.02386866725726913
412 BR(91) + HO2(14) <=> HBR(93) + O2(4) 0.015152354764004504
717 H(9) + HBR(93) <=> BR(91) + H2(11) 0.017840354082740966
90 CH3(20) + O(10) <=> CH2O(21) + H(9) 0.033385045